In [24]:
from typing import Dict, List, Any, Hashable, Tuple, Set
from collections import defaultdict
from datetime import datetime
import pandas as pd
import numpy as np
import json

from models import StudyResult
from utils import generate_key
from config import SCALAR_FIELDS, NON_SCALAR_FIELDS


In [25]:
EXPECTED_TABLES = StudyResult.expected_tables()

In [26]:
def process_study_file(file_loc: str) -> List[StudyResult]:

    batch_results: List[StudyResult] = []
    
    df_studies = pd.read_parquet(file_loc)
    df_studies = pd.json_normalize(df_studies["studies"])
    
    for idx, study in df_studies.iterrows():
        nct_index = SCALAR_FIELDS["nct_id"]
        nct_id = study.get(nct_index)
    
        if not nct_id:
            continue
    
        try:
            result = transform_single_study(nct_id, study)
            batch_results.append(result)
            
        except Exception as e:
            raise
    
    return batch_results

In [27]:
def merge_batch_results(batch_results: List[StudyResult]) -> Dict[str, List[Dict]]:
    
    merged: Dict[str, List[Dict]] = defaultdict(list)
    # print(f" LEN BATCH BEFORE MERGE ---{len(batch_results)}")
    # print(f" TYPE BATCH Before MERGE ---{type(batch_results)}")

    # print(type(batch_results[0].tables()))
    # print("-----")
    print(f"{type(batch_results)}")
    print(f"{len(batch_results)}")
    for study_result in batch_results:
        for table, rows in study_result.tables().items():
            merged[table].extend(rows)

    missing = set(EXPECTED_TABLES) - merged.keys()
    if missing:
        raise ValueError(f"Missing tables: {missing}")


    return merged


In [28]:
def transform_single_study(nct_id: str, study: pd.Series) -> StudyResult:
  
    study_key = generate_key(nct_id)
    result = defaultdict(list)

    # scalar fields
    study_fields = transform_scalar_fields(study_key, study)
    result["studies"].append(study_fields)

    # identificationModule
    secondary_ids, nct_aliases = transform_identification_module(study_key, study)
    result["secondary_ids"].extend(secondary_ids)
    result["nct_aliases"].extend(nct_aliases)

    # sponsorCollaboratorsModule
    sponsor, study_sponsor, collaborators, study_collaborators = (
        transform_sponsor_and_collaborators_module(nct_id, study_key, study)
    )
    result["sponsors"].extend(sponsor)
    result["study_sponsors"].extend(study_sponsor)
    result["collaborators"].extend(collaborators)
    result["study_collaborators"].extend(study_collaborators)

    # conditionsModule
    conditions, study_conditions, keywords, study_keywords = (
        transform_conditions_module(nct_id, study_key, study)
    )
    result["conditions"].extend(conditions)
    result["study_conditions"].extend(study_conditions)
    result["keywords"].extend(keywords)
    result["study_keywords"].extend(study_keywords)

    # armsInterventionsModule
    (
        arm_groups,
        arm_interventions,
        interventions,
        study_interventions,
        other_intervention_names,
        study_intervention_aliases,
    ) = transform_arms_interventions_module(study_key, study)
    result["arm_groups"].extend(arm_groups)
    result["arm_interventions"].extend(arm_interventions)
    result["interventions"].extend(interventions)
    result["study_interventions"].extend(study_interventions)
    result["other_intervention_names"].extend(other_intervention_names)
    result["study_intervention_aliases"].extend(study_intervention_aliases)

    # outcomesModule
    primary_outcomes, secondary_outcomes, other_outcomes = transform_outcomes_module(
        study_key, study
    )
    result["primary_outcomes"].extend(primary_outcomes)
    result["secondary_outcomes"].extend(secondary_outcomes)
    result["other_outcomes"].extend(other_outcomes)

    # contactsLocationsModule
    central_contacts, study_central_contacts, locations, study_locations = (
        transform_contacts_location_module(study_key, study)
    )
    result["central_contacts"].extend(central_contacts)
    result["study_central_contacts"].extend(study_central_contacts)
    result["locations"].extend(locations)
    result["study_locations"].extend(study_locations)

    # referencesModule
    references, link_references, ipd_references = transform_reference_module(
        study_key, study
    )
    result["references"].extend(references)
    result["link_references"].extend(link_references)
    result["ipd_references"].extend(ipd_references)

    # outcomeMeasuresModule
    (
        outcome_measures,
        outcome_measure_groups,
        outcome_measure_denom_units,
        outcome_measure_denom_counts,
        outcome_measure_groups_result,
        outcome_measure_analyses,
        outcome_measure_comparison_groups,
    ) = transform_outcome_measures_module(study_key, study)
    result["outcome_measures"].extend(outcome_measures)
    result["outcome_measure_groups"].extend(outcome_measure_groups)
    result["outcome_measure_denom_units"].extend(outcome_measure_denom_units)
    result["outcome_measure_denom_counts"].extend(outcome_measure_denom_counts)
    result["outcome_measure_groups_result"].extend(outcome_measure_groups_result)
    result["outcome_measure_analyses"].extend(outcome_measure_analyses)
    result["outcome_measure_comparison_groups"].extend(
        outcome_measure_comparison_groups
    )

    # participantFlowModule
    (
        flow_groups,
        flow_periods,
        flow_period_milestones,
        flow_period_milestone_achievements,
        df_flow_period_withdrawals,
        flow_period_withdrawal_reasons,
    ) = transform_participant_flow_module(study_key, study)
    result["flow_groups"].extend(flow_groups)
    result["flow_periods"].extend(flow_periods)
    result["flow_period_milestones"].extend(flow_period_milestones)
    result["flow_period_milestone_achievements"].extend(
        flow_period_milestone_achievements
    )
    result["df_flow_period_withdrawals"].extend(df_flow_period_withdrawals)
    result["flow_period_withdrawal_reasons"].extend(flow_period_withdrawal_reasons)

    # adverseEventsModule
    (
        adverse_events,
        event_groups,
        serious_events,
        serious_event_stats,
        other_events,
        other_event_stats,
    ) = transform_adverse_events_module(study_key, study)
    result["adverse_events"].extend(adverse_events)
    result["event_groups"].extend(event_groups)
    result["serious_events"].extend(serious_events)
    result["serious_event_stats"].extend(serious_event_stats)
    result["other_events"].extend(other_events)
    result["other_event_stats"].extend(other_event_stats)

    # annotationModule
    violations = transform_annotations_module(study_key, study)
    result["violations"].extend(violations)

    # conditionBrowseModule
    (
        conditions_mesh,
        study_conditions_mesh,
        conditions_mesh_ancestors,
        study_conditions_mesh_ancestors,
        conditions_browse_leaves,
        study_conditions_browse_leaves,
        conditions_browse_branches,
        study_conditions_browse_branches,
    ) = transform_conditions_browse_module(study_key, study)

    result["conditions_mesh"].extend(conditions_mesh)
    result["study_conditions_mesh"].extend(study_conditions_mesh)
    result["conditions_mesh_ancestors"].extend(conditions_mesh_ancestors)
    result["study_conditions_mesh_ancestors"].extend(study_conditions_mesh_ancestors)
    result["conditions_browse_leaves"].extend(conditions_browse_leaves)
    result["study_conditions_browse_leaves"].extend(study_conditions_browse_leaves)
    result["conditions_browse_branches"].extend(conditions_browse_branches)
    result["study_conditions_browse_branches"].extend(study_conditions_browse_branches)

    # interventionBrowseModule
    (
        interventions_mesh,
        study_interventions_mesh,
        interventions_mesh_ancestors,
        study_interventions_mesh_ancestors,
        interventions_browse_leaves,
        study_interventions_browse_leaves,
        interventions_browse_branches,
        study_interventions_browse_branches,
    ) = transform_interventions_browse_module(study_key, study)

    result["interventions_mesh"].extend(interventions_mesh)
    result["study_interventions_mesh"].extend(study_interventions_mesh)
    result["interventions_mesh_ancestors"].extend(interventions_mesh_ancestors)
    result["study_interventions_mesh_ancestors"].extend(
        study_interventions_mesh_ancestors
    )
    result["interventions_browse_leaves"].extend(interventions_browse_leaves)
    result["study_interventions_browse_leaves"].extend(
        study_interventions_browse_leaves
    )
    result["interventions_browse_branches"].extend(interventions_browse_branches)
    result["study_interventions_browse_branches"].extend(
        study_interventions_browse_branches
    )

    return StudyResult(
        studies=result["studies"],
        secondary_ids=result["secondary_ids"],
        nct_aliases=result["nct_aliases"],
        sponsors=result["sponsors"],
        study_sponsors=result["study_sponsors"],
        collaborators=result["collaborators"],
        study_collaborators=result["study_collaborators"],
        conditions=result["conditions"],
        study_conditions=result["study_conditions"],
        keywords=result["keywords"],
        study_keywords=result["study_keywords"],
        arm_groups=result["arm_groups"],
        arm_interventions=result["arm_interventions"],
        interventions=result["interventions"],
        study_interventions=result["study_interventions"],
        other_intervention_names=result["other_intervention_names"],
        study_intervention_aliases=result["study_intervention_aliases"],
        primary_outcomes=result["primary_outcomes"],
        secondary_outcomes=result["secondary_outcomes"],
        other_outcomes=result["other_outcomes"],
        central_contacts=result["central_contacts"],
        study_central_contacts=result["study_central_contacts"],
        locations=result["locations"],
        study_locations=result["study_locations"],
        references=result["references"],
        link_references=result["link_references"],
        ipd_references=result["ipd_references"],
        outcome_measures=result["outcome_measures"],
        outcome_measure_groups=result["outcome_measure_groups"],
        outcome_measure_denom_units=result["outcome_measure_denom_units"],
        outcome_measure_denom_counts=result["outcome_measure_denom_counts"],
        outcome_measure_groups_result=result["outcome_measure_groups_result"],
        outcome_measure_analyses=result["outcome_measure_analyses"],
        outcome_measure_comparison_groups=result["outcome_measure_comparison_groups"],
        flow_groups=result["flow_groups"],
        flow_periods=result["flow_periods"],
        flow_period_milestones=result["flow_period_milestones"],
        flow_period_milestone_achievements=result["flow_period_milestone_achievements"],
        df_flow_period_withdrawals=result["df_flow_period_withdrawals"],
        flow_period_withdrawal_reasons=result["flow_period_withdrawal_reasons"],
        adverse_events=result["adverse_events"],
        event_groups=result["event_groups"],
        serious_events=result["serious_events"],
        serious_event_stats=result["serious_event_stats"],
        other_events=result["other_events"],
        other_event_stats=result["other_event_stats"],
        violations=result["violations"],
        conditions_mesh=result["conditions_mesh"],
        study_conditions_mesh=result["study_conditions_mesh"],
        conditions_mesh_ancestors=result["conditions_mesh_ancestors"],
        study_conditions_mesh_ancestors=result["study_conditions_mesh_ancestors"],
        conditions_browse_leaves=result["conditions_browse_leaves"],
        study_conditions_browse_leaves=result["study_conditions_browse_leaves"],
        conditions_browse_branches=result["conditions_browse_branches"],
        study_conditions_browse_branches=result["study_conditions_browse_branches"],
        interventions_mesh=result["interventions_mesh"],
        study_interventions_mesh=result["study_interventions_mesh"],
        interventions_mesh_ancestors=result["interventions_mesh_ancestors"],
        study_interventions_mesh_ancestors=result["study_interventions_mesh_ancestors"],
        interventions_browse_leaves=result["interventions_browse_leaves"],
        study_interventions_browse_leaves=result["study_interventions_browse_leaves"],
        interventions_browse_branches=result["interventions_browse_branches"],
        study_interventions_browse_branches=result[
            "study_interventions_browse_branches"
        ]
    )


In [29]:
def transform_scalar_fields(study_key: str, study_data: pd.Series) -> Dict:
    study_record = dict()

    study_record["study_key"] = study_key
    for entity_key in SCALAR_FIELDS:
        index_field = SCALAR_FIELDS.get(entity_key)

        study_record[entity_key] = study_data.get(index_field)

    return study_record

In [30]:
def transform_identification_module(study_key: str, study_data: pd.Series) -> Tuple:
    secondary_ids = []
    nct_aliases = []

    identification_index = NON_SCALAR_FIELDS["identification"]["index_field"]

    # Secondary id infos
    secondary_id_infos = study_data.get(f"{identification_index}.secondaryIdInfos")

    if (
        isinstance(secondary_id_infos, (list, np.ndarray))
        and len(secondary_id_infos) > 0
    ):
        for secondary_id_info in secondary_id_infos:
            secondary_id = secondary_id_info.get("id")
            secondary_id_key = generate_key(study_key, secondary_id)

            secondary_ids.append(
                {
                    "secondary_id_key": secondary_id_key,
                    "study_key": study_key,
                    "id": secondary_id,
                    "type": secondary_id_info.get("type"),
                    "domain": secondary_id_info.get("domain"),
                    "link": secondary_id_info.get("link"),
                }
            )

    # nct id aliases
    nct_id_aliases = study_data.get(f"{identification_index}.nctIdAliases")

    if isinstance(nct_id_aliases, (list, np.ndarray)) and len(nct_id_aliases) > 0:
        for nct_id_alias in nct_id_aliases:
            nct_aliases.append(
                {
                    "study_key": study_key,
                    "id_alias": nct_id_alias,
                }
            )

    return secondary_ids, nct_aliases



In [31]:

def transform_sponsor_and_collaborators_module(
    nct_id: str, study_key: str, study_data: pd.Series
) -> Tuple:
    sponsor = []
    study_sponsor = []
    collaborators = []
    study_collaborators = []

    sponsor_collaborator_index = NON_SCALAR_FIELDS["sponsor_collaborators"][
        "index_field"
    ]

    ## sponsor name and class are scalar values and MUST be transformed as so
    lead_sponsor_name = study_data.get(f"{sponsor_collaborator_index}.leadSponsor.name")
    lead_sponsor_class = study_data.get(
        f"{sponsor_collaborator_index}.leadSponsor.class"
    )

    if pd.notna(lead_sponsor_name) and pd.notna(lead_sponsor_class):

        sponsor_key = generate_key(lead_sponsor_name, lead_sponsor_class)
        sponsor.append(
            {
                "sponsor_key": sponsor_key,
                "name": lead_sponsor_name,
                "sponsor_class": lead_sponsor_class,
            }
        )

        study_sponsor.append({"study_key": study_key, "sponsor_key": sponsor_key})

    # collaborators
    collaborators_list = study_data.get(
        f"{sponsor_collaborator_index}.collaborators"
    )

    if (
        isinstance(collaborators_list, (list, np.ndarray))
        and len(collaborators_list) > 0
    ):
        for collaborator in collaborators_list:
            collaborator_key = generate_key(
                collaborator.get("name"), collaborator.get("class")
            )

            collaborators.append(
                {
                    "collaborator_key": collaborator_key,
                    "name": collaborator.get("name"),
                    "collaborator_class": collaborator.get("class"),
                }
            )

            study_collaborators.append(
                {
                    "study_key": study_key,
                    "collaborator_key": collaborator_key,
                }
            )


    return sponsor, study_sponsor, collaborators, study_collaborators

          


In [32]:

def transform_conditions_module(
    nct_id: str, study_key: str, study_data: pd.Series
) -> Tuple:
    
    conditions = []
    study_conditions = []
    keywords = []
    study_keywords = []

    conditions_index = NON_SCALAR_FIELDS["conditions"]["index_field"]

    conditions_list = study_data.get(f"{conditions_index}.conditions")

    if isinstance(conditions_list, (list, np.ndarray)) and len(conditions_list) > 0:
        for condition in conditions_list:
            condition_key = generate_key(condition)

            conditions.append(
                {"condition_key": condition_key, "condition_name": condition.lower()}
            )

            study_conditions.append(
                {
                    "study_key": study_key,
                    "condition_key": condition_key,
                }
            )

    keywords_list = study_data.get(f"{conditions_index}.keywords")

    if isinstance(keywords_list, (list, np.ndarray)) and len(keywords_list) > 0:

        for keyword in keywords_list:
            keyword_key = generate_key(keyword)

            keywords.append({"keyword_key": keyword_key, "keyword_name": keyword.lower()})

            study_keywords.append(
                {
                    "study_key": study_key,
                    "keyword_key": keyword_key,
                }
            )

    return conditions, study_conditions, keywords, study_keywords


In [33]:


ALIASES = {
    "sham": "Placebo",
    "5-fu": "Fluorouracil",
    "5-fluorouracil": "Fluorouracil",
    "trastuzumab emtansine": "Trastuzumab",
    "t-dm1": "Trastuzumab",
}


ARMS_SOURCE = "ARM"


def standardize_intervention_name(
    intervention_name: str, source: str | None = None
) -> str | None:
    # Manual cleaning is not enough to handle randomness and unpredictability of the intervention names
    # as they're entered manually without validation on the API therefore:
    # MeSH terms are more reliable for queries but intervention names takes precedence over mesh during
    # API search as they're more specific and layman friendly

    if not intervention_name:
        return None

    if source == ARMS_SOURCE:
        # armGroups[].interventionNames uses format "Type: Name" (e.g., "Drug: Cisplatin")
        # interventions[].name uses just "Name" (e.g., "Cisplatin")
        # Stripping prefix is necessary to enable joining arm_interventions -> interventions

        parts = intervention_name.split(": ", 1)
        intervention_name = parts[1] if len(parts) > 1 else intervention_name

    cleaned = intervention_name.lower().strip()
    if cleaned in ALIASES:
        return ALIASES[cleaned]

    if cleaned.startswith("placebo"):
        # extensive descriptions of placebo arms e.g 'Placebo matched to M2951'
        # should only be in the description field in arms_intervention list
        return "Placebo"

    words = intervention_name.strip().split()
    return " ".join([w if w.isupper() else w.capitalize() for w in words])


def transform_arms_interventions_module(study_key: str, study_data: pd.Series) -> Tuple:

    arm_groups = []
    arm_interventions = []

    interventions = []
    study_interventions = []
    other_intervention_names = []
    study_intervention_aliases = []

    arms_interventions_index = NON_SCALAR_FIELDS["arms_interventions"]["index_field"]
    arm_groups_list = study_data.get(f"{arms_interventions_index}.armGroups")

    if isinstance(arm_groups_list, (list, np.ndarray)) and len(arm_groups_list) > 0:
        for arm_group in arm_groups_list:
            arm_label = arm_group.get("label")
            arm_description = arm_group.get("description")
            arm_type = arm_group.get("type")

            arm_group_key = generate_key(
                study_key, arm_label, arm_description, arm_type
            )

            arm_groups.append(
                {
                    "study_key": study_key,
                    "arm_group_key": arm_group_key,
                    "arm_label": arm_label,
                    "arm_description": arm_description,
                    "arm_type": arm_type,
                }
            )

            arm_interventions_list = arm_group.get("interventionNames")
            if (
                isinstance(arm_interventions_list, (list, np.ndarray))
                and len(arm_interventions_list) > 0
            ):

                for arm_intervention in arm_interventions_list:
                    arm_intervention_name = standardize_intervention_name(
                        arm_intervention, ARMS_SOURCE
                    )
                    arm_intervention_key = generate_key(arm_intervention_name)

                    arm_interventions.append(
                        {
                            "study_key": study_key,
                            "arm_group_key": arm_group_key,
                            "arm_intervention_key": arm_intervention_key,
                            "arm_intervention_name": arm_intervention_name,
                        }
                    )

    interventions_list = study_data.get(f"{arms_interventions_index}.interventions")
    if (
        isinstance(interventions_list, (list, np.ndarray))
        and len(interventions_list) > 0
    ):
        for intervention in interventions_list:
            main_name = standardize_intervention_name(intervention.get("name"))
            intervention_type = intervention.get("type")

            intervention_key = generate_key(
                main_name
            )  # Only name is used to enable matching on both arms and interventions

            interventions.append(
                {
                    "intervention_key": intervention_key,
                    "intervention_name": main_name,
                    "intervention_type": intervention_type,
                }
            )

            study_interventions.append(
                {
                    "study_key": study_key,
                    "intervention_key": intervention_key,
                    "description": intervention.get("description"),
                    "is_primary_name": True,
                }
            )

            other_names = intervention.get("otherNames")

            if isinstance(other_names, (list, np.ndarray)) and len(other_names) > 0:
                for other_name in other_names:
                    other_name = standardize_intervention_name(other_name)
                    if other_name == main_name:
                        continue  # some studies put the main name in the list of other names

                    other_intervention_key = generate_key(other_name)
                    other_intervention_names.append(
                        {
                            "intervention_key": other_intervention_key,
                            # has its own key as other here could be main and vice versa in other studies.
                            # and it remains independent in the warehouse
                            "intervention_name": other_name,
                            "intervention_type": intervention_type,  # inherit from parent
                        }
                    )

                    study_intervention_aliases.append(
                        {
                            "study_key": study_key,
                            "intervention_key": other_intervention_key,
                            "description": intervention.get("description"),
                            "is_primary_name": False,
                        }
                    )
            # armGroupLabels is excluded to avoid bi-directional inconsistencies due to human errors from source.
            # check documentation/excluded_fields.md for details

    return (
        arm_groups,
        arm_interventions,
        interventions,
        study_interventions,
        other_intervention_names,
        study_intervention_aliases,
    )


In [34]:

def transform_outcomes_module(study_key: str, study_data: pd.Series) -> Tuple:
 
    primary_outcomes = []
    secondary_outcomes = []
    other_outcomes = []

    outcomes_module_index = NON_SCALAR_FIELDS["outcomes"]["index_field"]

    primary_outcomes_list = study_data.get(f"{outcomes_module_index}.primaryOutcomes")

    if (
        isinstance(primary_outcomes_list, (list, np.ndarray))
        and len(primary_outcomes_list) > 0
    ):
        for primary_outcome in primary_outcomes_list:
            outcome_measure = primary_outcome.get("measure")
            outcome_description = primary_outcome.get("description")
            outcome_timeframe = primary_outcome.get("timeFrame")
            
            outcome_key = generate_key(study_key, outcome_measure, outcome_description, outcome_timeframe)

            primary_outcomes.append(
                {
                    "study_key": study_key,
                    "outcome_key": outcome_key,
                    "measure": outcome_measure,
                    "description": outcome_description,
                    "time_frame": outcome_timeframe,
                }
            )

    secondary_outcomes_list = study_data.get(
        f"{outcomes_module_index}.secondaryOutcomes"
    )
    if (
        isinstance(secondary_outcomes_list, (list, np.ndarray))
        and len(secondary_outcomes_list) > 0
    ):
        for secondary_outcome in secondary_outcomes_list:
            outcome_measure = secondary_outcome.get("measure")
            outcome_description = secondary_outcome.get("description")
            outcome_timeframe = secondary_outcome.get("timeFrame")
            
            outcome_key = generate_key(study_key, outcome_measure, outcome_description, outcome_timeframe)

            secondary_outcomes.append(
                {
                    "study_key": study_key,
                    "outcome_key": outcome_key,
                    "measure": outcome_measure,
                    "description": outcome_description,
                    "time_frame": outcome_timeframe,
                }
            )
            

    other_outcomes_list = study_data.get(f"{outcomes_module_index}.otherOutcomes")
    if (
        isinstance(other_outcomes_list, (list, np.ndarray))
        and len(other_outcomes_list) > 0
    ):
        for other_outcome in other_outcomes_list:
            outcome_measure = other_outcome.get("measure")
            outcome_description = other_outcome.get("description")
            outcome_timeframe = other_outcome.get("timeFrame")

            outcome_key = generate_key(study_key, outcome_measure, outcome_description, outcome_timeframe)

            other_outcomes.append(
                {
                    "study_key": study_key,
                    "outcome_key": outcome_key,
                    "measure": outcome_measure,
                    "description": outcome_description,
                    "time_frame": outcome_timeframe,
                }
            )

    return primary_outcomes, secondary_outcomes, other_outcomes


In [35]:

def transform_contacts_location_module(study_key: str, study_data: pd.Series) -> Tuple:
    central_contacts = []
    study_central_contacts = []
    locations = []
    study_locations = []

    contacts_locations_index = NON_SCALAR_FIELDS["contacts_location"]["index_field"]

    #contacts
    central_contacts_list = study_data.get(f"{contacts_locations_index}.centralContacts")


    if (
        isinstance(central_contacts_list, (list, np.ndarray))
        and len(central_contacts_list) > 0
    ):

        for central_contact in central_contacts_list:
            name = central_contact.get("name")
            role = central_contact.get("role")
            phone = central_contact.get("phone")
            email = central_contact.get("email")

            central_contact_key = generate_key(name, role, phone, email)

            central_contacts.append(
                {
                    "contact_key": central_contact_key,
                    "name": name,
                    "role": role,
                    "phone": phone,
                    "email": email,
                }
            )
            
            study_central_contacts.append(
                {
                    "study_key": study_key,
                    "contact_key": central_contact_key,
                }
            )

    #locations
    locations_list = study_data.get(f"{contacts_locations_index}.locations")

    if isinstance(locations_list, (list, np.ndarray)) and len(locations_list) > 0:
        for location in locations_list:
            facility = location.get("facility")
            city = location.get("city")
            state = location.get("state")
            country = location.get("country")

            location_key = generate_key(facility, city, state, country)
            curr_location = {
                "location_key": location_key,
                "facility": facility,
                "city": city,
                "state": state,
                "country": country
            }
            geopoint = location.get("geoPoint")

            if isinstance(geopoint, dict) and geopoint:
                curr_location["lat"] = (
                    float(geopoint.get("lat")) if geopoint.get("lat") else None
                )
                curr_location["lon"] = (
                    float(geopoint.get("lon")) if geopoint.get("lon") else None
                )

            locations.append(curr_location)

            study_locations.append(
                {
                    "study_key": study_key,
                    "location_key": location_key,
                    "status": location.get("status"),
                    "contacts": location.get("contacts"), #json blob
                }
            )

    return central_contacts, study_central_contacts, locations, study_locations



In [36]:



def transform_reference_module(study_key: str, study_data: pd.Series) -> Tuple:
    study_references = []
    link_references = []
    ipd_references = []

    references_index = NON_SCALAR_FIELDS["references"]["index_field"]

    references_list = study_data.get(f"{references_index}.references")

    if isinstance(references_list, (list, np.ndarray)) and len(references_list) > 0:

        for reference in references_list:
            pmid = reference.get("pmid")
            reference_key = generate_key(study_key, pmid)
            study_references.append(
                {
                    "study_key": study_key,
                    "ref_key": reference_key,
                    "pmid": pmid,
                    "type": reference.get("type"),
                    "citation": reference.get("citation"),
                }
            )

    links_list = study_data.get(f"{references_index}.seeAlsoLinks")
    if isinstance(links_list, (list, np.ndarray)) and len(links_list) > 0:

        for link in links_list:
            label = link.get("label")
            url = link.get("url")
            link_key = generate_key(study_key, label, url)
            link_references.append(
                {
                    "study_key": study_key,
                    "link_key": link_key,
                    "label": label,
                    "url": url
                }
            )

    ipds_list = study_data.get(f"{references_index}.availIpds")

    if isinstance(ipds_list, (list, np.ndarray)) and len(ipds_list) > 0:

        for ipd in ipds_list:
            ipd_id = ipd.get("id")
            ipd_type = ipd.get("type")
            ipd_url = ipd.get("url")

            ipd_key = generate_key(study_key, ipd_id, ipd_type, ipd_url)
            ipd_references.append(
                {
                    "study_key": study_key,
                    "ipd_key": ipd_key,
                    "id": ipd_id,
                    "type": ipd_type,
                    "url": ipd_url,
                    "comment": ipd.get("comment"),
                }
            )

    return study_references, link_references, ipd_references


In [37]:


def transform_outcome_measures_module(study_key: str, study_data: pd.Series) -> Tuple:
    outcome_measures = []
    outcome_measure_groups = []
    outcome_measure_denom_units = []
    outcome_measure_denom_counts = []
    outcome_measure_groups_result = []
    outcome_measure_analyses = []
    outcome_measure_comparison_groups = []

    outcomes_index = NON_SCALAR_FIELDS["outcome_measures"]["index_field"]
    outcomes_measures_list = study_data.get(f"{outcomes_index}.outcomeMeasures")

    if (
        isinstance(outcomes_measures_list, (list, np.ndarray))
        and len(outcomes_measures_list) > 0
    ):
        for outcome_measure in outcomes_measures_list:
            outcome_measure_title = outcome_measure.get("title")
            outcome_measure_type = outcome_measure.get("type")
            outcome_measure_key = generate_key(
                study_key, outcome_measure_title, outcome_measure_type
            )

            outcome_measures.append(
                {
                    "outcome_measure_key": outcome_measure_key,
                    "study_key": study_key,
                    "outcome_type": outcome_measure_type,
                    "title": outcome_measure_title,
                    "description": outcome_measure.get("description"),
                    "population_description": outcome_measure.get(
                        "populationDescription"
                    ),
                    "reporting_status": outcome_measure.get("reportingStatus"),
                    "anticipated_posting_date": outcome_measure.get(
                        "anticipatedPostingDate"
                    ),
                    "param_type": outcome_measure.get("paramType"),
                    "dispersion_type": outcome_measure.get("dispersionType"),
                    "unit_of_measure": outcome_measure.get("unitOfMeasure"),
                    "calculate_pct": outcome_measure.get("calculatePct"),
                    "time_frame": outcome_measure.get("timeFrame"),
                    "denom_units_selected": outcome_measure.get("denomUnitsSelected"),
                }
            )

            groups = outcome_measure.get("groups")
            if isinstance(groups, (list, np.ndarray)) and len(groups) > 0:
                for group in groups:
                    group_id = group.get("id")
                    outcome_group_key = generate_key(
                        study_key, outcome_measure_key, group_id
                    )

                    outcome_measure_groups.append(
                        {
                            "group_key": outcome_group_key,
                            "study_key": study_key,
                            "outcome_measure_key": outcome_measure_key,
                            "group_id": group_id,
                            "title": group.get("title"),
                            "description": group.get("description"),
                        }
                    )

            denoms_list = outcome_measure.get("denoms")
            if isinstance(denoms_list, (list, np.ndarray)) and len(denoms_list) > 0:
                for denom in denoms_list:
                    denom_unit = denom.get("units")
                    denom_unit = denom_unit.upper() if denom_unit else "UNKNOWN"
                    denom_unit_key = generate_key(denom_unit)

                    outcome_measure_denom_units.append(
                        {
                            "denom_unit_key": denom_unit_key,
                            "denom_unit": denom_unit,
                        }
                    )

                    denom_counts = denom.get("counts")
                    if (
                        isinstance(denom_counts, (list, np.ndarray))
                        and len(denom_counts) > 0
                    ):
                        for denom_count in denom_counts:
                            denom_count_group_id = denom_count.get("groupId")
                            denom_count_group_key = generate_key(
                                study_key, outcome_measure_key, denom_count_group_id
                            )
                            denom_count_key = generate_key(
                                study_key,
                                outcome_measure_key,
                                denom_unit_key,
                                denom_count_group_id,
                            )

                            outcome_measure_denom_counts.append(
                                {
                                    "denom_count_key": denom_count_key,
                                    "study_key": study_key,
                                    "outcome_measure_key": outcome_measure_key,
                                    "denom_unit_key": denom_unit_key,
                                    "group_key": denom_count_group_key,
                                    "group_id": denom_count_group_id,
                                    "denom_value": denom_count.get("value"),
                                }
                            )

            classes = outcome_measure.get("classes")
            if isinstance(classes, (list, np.ndarray)) and len(classes) > 0:
                for single_class in classes:
                    # class is not a true container and only wraps categories contrary to what the docs say
                    categories = single_class.get("categories")

                    if (
                        isinstance(categories, (list, np.ndarray))
                        and len(categories) > 0
                    ):
                        for cat_idx, category in enumerate(categories):
                            measurements = category.get("measurements")
                            if (
                                isinstance(measurements, (list, np.ndarray))
                                and len(measurements) > 0
                            ):
                                for measurement in measurements:
                                    meas_group_id = measurement.get("groupId")
                                    meas_group_key = generate_key(
                                        study_key, outcome_measure_key, meas_group_id
                                    )  # group keys must be created the same way

                                    outcome_measure_groups_result.append(
                                        {
                                            "group_key": meas_group_key,
                                            "outcome_measure_key": outcome_measure_key,
                                            "study_key": study_key,
                                            "group_id": meas_group_id,
                                            "value": measurement.get("value"),
                                            "lower_limit": measurement.get(
                                                "lowerLimit"
                                            ),
                                            "upper_limit": measurement.get(
                                                "upperLimit"
                                            ),
                                            "spread": measurement.get("spread"),
                                            "comment": measurement.get("comment"),
                                        }
                                    )

            analyses = outcome_measure.get("analyses")
            if isinstance(analyses, (list, np.ndarray)) and len(analyses) > 0:
                for analysis in analyses:
                    param_type = analysis.get("paramType")
                    param_value = analysis.get("paramValue")

                    analysis_key = generate_key(
                        study_key, outcome_measure_key, param_type, param_value
                    )

                    outcome_measure_analyses.append(
                        {
                            "analysis_key": analysis_key,
                            "study_key": study_key,
                            "outcome_measure_key": outcome_measure_key,
                            "param_type": param_type,
                            "param_value": param_value,
                            "dispersion_type": analysis.get("dispersionType"),
                            "dispersion_value": analysis.get("dispersionValue"),
                            "statistical_method": analysis.get("statisticalMethod"),
                            "statistical_comment": analysis.get("statisticalComment"),
                            "p_value": analysis.get("pValue"),
                            "p_value_comment": analysis.get("pValueComment"),
                            "non_inferiority_type": analysis.get("nonInferiorityType"),
                        }
                    )

                    analysis_comparison_groups = analysis.get("groupIds")
                    if (
                        isinstance(analysis_comparison_groups, (list, np.ndarray))
                        and len(analysis_comparison_groups) > 0
                    ):
                        for group_id in analysis_comparison_groups:
                            group_key = generate_key(
                                study_key, outcome_measure_key, group_id
                            )
                            outcome_measure_comparison_groups.append(
                                {   "study_key": study_key,
                                    "outcome_measure_key": outcome_measure_key,
                                    "analysis_key": analysis_key,
                                    "group_key": group_key,
                                    "group_id": group_id,
                                }
                            )

    return (
        outcome_measures,
        outcome_measure_groups,
        outcome_measure_denom_units,
        outcome_measure_denom_counts,
        outcome_measure_groups_result,
        outcome_measure_analyses,
        outcome_measure_comparison_groups,
    )


In [38]:


def transform_participant_flow_module(study_key: str, study_data: pd.Series) -> Tuple:
 
    flow_groups = []
    flow_periods = []
    flow_period_milestones = []
    flow_period_milestone_achievements = []
    flow_period_withdrawals = []
    flow_period_withdrawal_reasons = []

    flow_module_index = NON_SCALAR_FIELDS["participant_flow"]["index_field"]

    # flow groups
    flow_group_list = study_data.get(f"{flow_module_index}.groups")
    if isinstance(flow_group_list, (list, np.ndarray)) and len(flow_group_list) > 0:
        for flow in flow_group_list:
            group_id = flow.get("id")
            group_key = generate_key(study_key, group_id)

            flow_groups.append(
                {
                    "study_key": study_key,
                    "group_key": group_key,
                    "id": group_id,
                    "title": flow.get("title"),
                    "description": flow.get("description"),
                }
            )

    # flow periods

    flow_period_list = study_data.get(f"{flow_module_index}.periods")
    if isinstance(flow_period_list, (list, np.ndarray)) and len(flow_period_list) > 0:
        for period in flow_period_list:
            period_title = period.get("title")
            period_key = generate_key(study_key, period_title)

            # flow periods
            flow_periods.append(
                {
                    "study_key": study_key,
                    "period_key": period_key,
                    "title": period.get("title"),
                }
            )

            # flow milestones
            period_milestones = period.get("milestones")
            if (
                isinstance(period_milestones, (list, np.ndarray))
                and len(period_milestones) > 0
            ):
                for period_milestone in period_milestones:
                    milestone_type = period_milestone.get("type")
                    milestone_comment = period_milestone.get("comment")
                    milestone_key = generate_key(
                        study_key, period_key, milestone_type, milestone_comment
                    )

                    flow_period_milestones.append(
                        {
                            "study_key": study_key,
                            "period_key": period_key,
                            "milestone_key": milestone_key,
                            "type": milestone_type,
                            "comment": milestone_comment,
                        }
                    )

                    # milestone achievements
                    milestone_achievements = period_milestone.get("achievements")
                    if (
                        isinstance(milestone_achievements, (list, np.ndarray))
                        and len(milestone_achievements) > 0
                    ):
                        for achievement in milestone_achievements:
                            group_id = achievement.get("groupId")
                            group_key = generate_key(study_key, group_id)
                            flow_period_milestone_achievements.append(
                                {
                                    "study_key": study_key,
                                    "period_key": period_key,
                                    "milestone_key": milestone_key,
                                    "group_key": group_key,
                                    "group_id": group_id,
                                    "comment": achievement.get("comment"),
                                    "num_subjects": achievement.get("numSubjects"),
                                    "num_units": achievement.get("numUnits"),
                                }
                            )

            period_withdrawals = period.get("dropWithdraws")
            if (
                isinstance(period_withdrawals, (list, np.ndarray))
                and len(period_withdrawals) > 0
            ):
                for withdrawal in period_withdrawals:
                    withdrawal_type = withdrawal.get("type")
                    withdrawal_comment = withdrawal.get("comment")
                    withdrawal_key = generate_key(
                        study_key, period_key, withdrawal_type, withdrawal_comment
                    )

                    flow_period_withdrawals.append(
                        {
                            "study_key": study_key,
                            "period_key": period_key,
                            "withdrawal_key": withdrawal_key,
                            "type": withdrawal_type,
                            "comment": withdrawal_comment,
                        }
                    )

                    # withdrawal reasons
                    period_withdrawal_reasons = withdrawal.get("reasons")

                    if (
                        isinstance(period_withdrawal_reasons, (list, np.ndarray))
                        and len(period_withdrawal_reasons) > 0
                    ):
                        for reason in period_withdrawal_reasons:

                            reason_group_id = reason.get("groupId")
                            reason_comment = reason.get("comment")

                            reason_key = generate_key(
                                study_key,
                                period_key,
                                withdrawal_key,
                                reason_group_id,
                                reason_comment,
                            )

                            group_key = generate_key(study_key, reason_group_id)

                            flow_period_withdrawal_reasons.append(
                                {
                                    "study_key": study_key,
                                    "period_key": period_key,
                                    "withdrawal_key": withdrawal_key,
                                    "reason_key": reason_key,
                                    "group_id": reason_group_id,
                                    "group_key": group_key,
                                    "reason": reason_comment,
                                    "num_subjects": reason.get("numSubjects"),
                                }
                            )

    return (
        flow_groups,
        flow_periods,
        flow_period_milestones,
        flow_period_milestone_achievements,
        flow_period_withdrawals,
        flow_period_withdrawal_reasons,
    )


In [39]:


def transform_adverse_events_module(study_key: str, study_data: pd.Series) -> Tuple:
   
    adverse_events = []
    event_groups = []
    serious_events = []
    serious_event_stats = []
    other_events = []
    other_event_stats = []

    events_index = NON_SCALAR_FIELDS["adverse_events"]["index_field"]

    description = study_data.get(f"{events_index}.description")
    adverse_event_key = generate_key(study_key, description)

    # AE fields are scalar
    adverse_events.append(
        {
            "adverse_event_key": adverse_event_key,
            "study_key": study_key,
            "description": description,
            "frequency_threshold": study_data.get(f"{events_index}.frequencyThreshold"),
            "time_frame": study_data.get(f"{events_index}.timeFrame"),
            "mortality_cmt": study_data.get(f"{events_index}.allCauseMortalityComment"),
        }
    )

    event_group_list = study_data.get(f"{events_index}.eventGroups")

    if isinstance(event_group_list, (list, np.ndarray)) and len(event_group_list) > 0:
        for event_group in event_group_list:
            group_id = event_group.get("id")
            event_group_key = generate_key(study_key, adverse_event_key, group_id)

            event_groups.append(
                {
                    "event_group_key": event_group_key,
                    "study_key": study_key,
                    "adverse_event_key": adverse_event_key,
                    "group_id": group_id,
                    "title": event_group.get("title"),
                    "description": event_group.get("description"),
                    "num_deaths": event_group.get("deathsNumAffected"),
                    "num_deaths_at_risk": event_group.get("deathsNumAtRisk"),
                    "num_serious": event_group.get("seriousNumAffected"),
                    "num_serious_at_risk": event_group.get("seriousNumAtRisk"),
                    "num_other": event_group.get("otherNumAffected"),
                    "num_other_at_risk": event_group.get("otherNumAtRisk"),
                }
            )
    # serious AEs
    serious_events_list = study_data.get(f"{events_index}.seriousEvents")

    if isinstance(serious_events_list, (list, np.ndarray)) and len(serious_events_list) > 0:
        for serious_event in serious_events_list:
            term = serious_event.get("term")
            serious_event_key = generate_key(study_key, adverse_event_key, term)

            serious_events.append(
                {
                    "serious_event_key": serious_event_key,
                    "adverse_event_key": adverse_event_key,
                    "study_key": study_key,
                    "term": term,
                    "organ_system": serious_event.get("organSystem"),
                    "source_vocab": serious_event.get("sourceVocabulary"),
                    "assessment_type": serious_event.get("assessmentType"),
                    "notes": serious_event.get("notes"),
                }
            )

            serious_event_stats_list = serious_event.get("stats")
            if (
                isinstance(serious_event_stats_list, (list, np.ndarray))
                and len(serious_event_stats_list) > 0
            ):
                for serious_event_stat in serious_event_stats_list:
                    group_id = serious_event_stat.get("groupId")

                    group_key = generate_key(study_key, adverse_event_key, group_id)
                    event_stat_key = generate_key(
                        study_key, adverse_event_key, serious_event_key, group_id
                    )

                    serious_event_stats.append(
                        {
                            "event_stat_key": event_stat_key,
                            "adverse_event_key": adverse_event_key,
                            "serious_event_key": serious_event_key,
                            "study_key": study_key,
                            "group_key": group_key,
                            "group_id": group_id,
                            "num_events": serious_event_stat.get("numEvents"),
                            "num_affected": serious_event_stat.get("numAffected"),
                            "num_at_risk": serious_event_stat.get("numAtRisk"),
                        }
                    )

    # other AEs
    other_events_list = study_data.get(f"{events_index}.otherEvents")
    if isinstance(other_events_list, (list, np.ndarray)) and len(other_events_list) > 0:
        for other_event in other_events_list:
            term = other_event.get("term")
            other_event_key = generate_key(study_key, adverse_event_key, term)

            other_events.append(
                {
                    "other_event_key": other_event_key,
                    "adverse_event_key": adverse_event_key,
                    "study_key": study_key,
                    "term": term,
                    "organ_system": other_event.get("organSystem"),
                    "source_vocab": other_event.get("sourceVocabulary"),
                    "assessment_type": other_event.get("assessmentType"),
                    "notes": other_event.get("notes"),
                }
            )

            other_event_stats_list = other_event.get("stats")
            if (
                isinstance(other_event_stats_list, (list, np.ndarray))
                and len(other_event_stats_list) > 0
            ):
                for other_event_stat in other_event_stats_list:
                    group_id = other_event_stat.get("groupId")

                    group_key = generate_key(study_key, adverse_event_key, group_id)
                    event_stat_key = generate_key(
                        study_key, adverse_event_key, other_event_key, group_id
                    )

                    other_event_stats.append(
                        {
                            "event_stat_key": event_stat_key,
                            "other_event_key": other_event_key,
                            "adverse_event_key": adverse_event_key,
                            "study_key": study_key,
                            "group_key": group_key,
                            "group_id": group_id,
                            "num_events": other_event_stat.get("numEvents"),
                            "num_affected": other_event_stat.get("numAffected"),
                            "num_at_risk": other_event_stat.get("numAtRisk"),
                        }
                    )

    return (
        adverse_events,
        event_groups,
        serious_events,
        serious_event_stats,
        other_events,
        other_event_stats,
    )


In [40]:

def transform_annotations_module(study_key: str, study_data: pd.Series) -> List:
    """
    Extract FDAAA 801 regulatory violations.
    These indicate FDA compliance issues (non-submission, false information, penalties).
    """
    violations = []

    annotations_index = NON_SCALAR_FIELDS["annotations"]["index_field"]
    violations_list  = study_data.get(annotations_index)

    if isinstance(violations_list, (list, np.ndarray)) and len(violations_list) > 0:

        for violation in violations_list:
            issued_date = violation.get("issuedDate")
            event_type = violation.get("type")
            violation_key = generate_key(study_key, issued_date, event_type)

            violations.append(
                {
                    "violation_key": violation_key,
                    "study_key": study_key,
                    "violation_type": event_type,
                    "issued_date": issued_date,
                    "description": violation.get("description"),
                    "creation_date": violation.get("creationDate"),
                    "release_date": violation.get("releaseDate"),
                    "posted_date": violation.get("postedDate"),
                }
            )

    return violations


In [41]:

def transform_conditions_browse_module(study_key: str, study_data: pd.Series) -> Tuple:
   
    conditions_mesh = []
    study_conditions_mesh = []

    conditions_mesh_ancestors = []
    study_conditions_mesh_ancestors = []

    conditions_browse_leaves = []
    study_conditions_browse_leaves = []

    conditions_browse_branches = []
    study_conditions_browse_branches = []

    conditions_browse_index = NON_SCALAR_FIELDS["conditions_browse"]["index_field"]

    meshes = study_data.get(f"{conditions_browse_index}.meshes")
    if isinstance(meshes, (list, np.ndarray)) and len(meshes) > 0:
        for mesh in meshes:
            mesh_terms = mesh.get("term")

            if isinstance(mesh_terms, str) and mesh_terms:
                terms = mesh_terms.split(",")
                for term in terms:
                    term = term.strip()
                    mesh_key = generate_key(term)
                    conditions_mesh.append(
                        {
                            "mesh_key": mesh_key,
                            "mesh_id": mesh.get("id"),
                            "mesh_term": term.lower(),
                        }
                    )

                    study_conditions_mesh.append(
                        {"mesh_key": mesh_key, "study_key": study_key}
                    )

    ancestors_list = study_data.get(f"{conditions_browse_index}.ancestors")
    if (
        isinstance(ancestors_list, (list, np.ndarray))
        and len(ancestors_list) > 0
    ):
        for ancestor in ancestors_list:
            ancestor_terms = ancestor.get("term")

            if isinstance(ancestor_terms, str) and ancestor_terms:
                terms = ancestor_terms.split(",") #sometimes MeSH terms are comma separated keywords
                for term in terms:
                    term = term.strip()
                    ancestor_key = generate_key(term)
                    conditions_mesh_ancestors.append(
                        {
                            "ancestor_key": ancestor_key,
                            "ancestor_id": ancestor.get("id"),
                            "term": term.lower(),
                        }
                    )

                    study_conditions_mesh_ancestors.append(
                        {"ancestor_key": ancestor_key, "study_key": study_key}
                    )

    mesh_browse_leaves = study_data.get(f"{conditions_browse_index}.browseLeaves")
    if (
        isinstance(mesh_browse_leaves, (list, np.ndarray))
        and len(mesh_browse_leaves) > 0
    ):
        for browse_leaf in mesh_browse_leaves:
            leaf_id = browse_leaf.get("id")
            leaf_key = generate_key(leaf_id)

            conditions_browse_leaves.append(
                {
                    "leaf_key": leaf_key,
                    "leaf_id": leaf_id,
                    "name": browse_leaf.get("name").lower(),
                    "as_found": browse_leaf.get("asFound"),
                    "relevance": browse_leaf.get("relevance"),
                }
            )

            study_conditions_browse_leaves.append(
                {"leaf_key": leaf_key, "study_key": study_key}
            )

    mesh_browse_branches = study_data.get(f"{conditions_browse_index}.browseBranches")

    if (
        isinstance(mesh_browse_branches, (list, np.ndarray))
        and len(mesh_browse_branches) > 0
    ):
        for browse_branch in mesh_browse_branches:
            branch_id = browse_branch.get("id")
            branch_key = generate_key(branch_id)

            conditions_browse_branches.append(
                {
                    "branch_key": branch_key,
                    "branch_id": branch_id,
                    "name": browse_branch.get("name").lower(),
                    "as_found": browse_branch.get("asFound"),
                    "relevance": browse_branch.get("relevance"),
                }
            )

            study_conditions_browse_branches.append(
                {"branch_key": branch_key, "study_key": study_key}
            )

    return (
        conditions_mesh,
        study_conditions_mesh,
        conditions_mesh_ancestors,
        study_conditions_mesh_ancestors,
        conditions_browse_leaves,
        study_conditions_browse_leaves,
        conditions_browse_branches,
        study_conditions_browse_branches,
    )


In [42]:

def transform_interventions_browse_module(
    study_key: str, study_data: pd.Series
) -> Tuple:

    interventions_mesh = []
    study_interventions_mesh = []

    interventions_mesh_ancestors = []
    study_interventions_mesh_ancestors = []

    interventions_browse_leaves = []
    study_interventions_browse_leaves = []

    interventions_browse_branches = []
    study_interventions_browse_branches = []

    interventions_browse_index = NON_SCALAR_FIELDS["interventions_browse"][
        "index_field"
    ]

    meshes = study_data.get(f"{interventions_browse_index}.meshes")
    if isinstance(meshes, (list, np.ndarray)) and len(meshes) > 0:
        for mesh in meshes:
            mesh_terms = mesh.get("term")

            if isinstance(mesh_terms, str) and mesh_terms:
                terms = mesh_terms.split(",")
                for term in terms:
                    term = term.strip()
                    mesh_key = generate_key(term)
                    interventions_mesh.append(
                        {
                            "mesh_key": mesh_key,
                            "mesh_id": mesh.get("id"),
                            "mesh_term": term.lower(),
                        }
                    )

                    study_interventions_mesh.append(
                        {"mesh_key": mesh_key, "study_key": study_key}
                    )

    ancestors_list = study_data.get(f"{interventions_browse_index}.ancestors")
    if (
        isinstance(ancestors_list, (list, np.ndarray))
        and len(ancestors_list) > 0
    ):

        for ancestor in ancestors_list:
            ancestor_terms = ancestor.get("term")

            if isinstance(ancestor_terms, str) and ancestor_terms:
                terms = ancestor_terms.split(",")
                for term in terms:
                    term = term.strip()
                    ancestor_key = generate_key(term)
                    interventions_mesh_ancestors.append(
                        {
                            "ancestor_key": ancestor_key,
                            "study_key": study_key,
                            "ancestor_id": ancestor.get("id"),
                            "ancestor_term": term.lower(),
                        }
                    )

                    study_interventions_mesh_ancestors.append(
                        {"ancestor_key": ancestor_key, "study_key": study_key}
                    )

    mesh_browse_leaves = study_data.get(f"{interventions_browse_index}.browseLeaves")
    if (
        isinstance(mesh_browse_leaves, (list, np.ndarray))
        and len(mesh_browse_leaves) > 0
    ):
        for browse_leaf in mesh_browse_leaves:
            leaf_id = browse_leaf.get("id")
            leaf_key = generate_key(leaf_id)

            interventions_browse_leaves.append(
                {
                    "leaf_key": leaf_key,
                    "name": browse_leaf.get("name").lower(),
                    "as_found": browse_leaf.get("asFound"),
                    "relevance": browse_leaf.get("relevance"),
                }
            )

            study_interventions_browse_leaves.append(
                {"leaf_key": leaf_key, "study_key": study_key}
            )

    mesh_browse_branches = study_data.get(
        f"{interventions_browse_index}.browseBranches"
    )
    if (
        isinstance(mesh_browse_branches, (list, np.ndarray))
        and len(mesh_browse_branches) > 0
    ):
        for browse_branch in mesh_browse_branches:
            branch_id = browse_branch.get("id")
            branch_key = generate_key(branch_id)

            interventions_browse_branches.append(
                {
                    "branch_key": branch_key,
                    "name": browse_branch.get("name").lower(),
                    "as_found": browse_branch.get("asFound"),
                    "relevance": browse_branch.get("relevance"),
                }
            )

            study_interventions_browse_branches.append(
                {"branch_key": branch_key, "study_key": study_key}
            )

    return (
        interventions_mesh,
        study_interventions_mesh,
        interventions_mesh_ancestors,
        study_interventions_mesh_ancestors,
        interventions_browse_leaves,
        study_interventions_browse_leaves,
        interventions_browse_branches,
        study_interventions_browse_branches,
    )


In [43]:

def post_process_tables(results: Dict[str, List[Dict]]) -> List[pd.DataFrame]:
 
    df_studies = pd.DataFrame(results["studies"])

    # identificationModule
    df_secondary_ids = pd.DataFrame(results["secondary_ids"])
    df_nct_aliases = pd.DataFrame(results["nct_aliases"])

    # sponsorCollaboratorsModule
    df_sponsors = pd.DataFrame(results["sponsors"])
    df_study_sponsors = pd.DataFrame(results["study_sponsors"])
    df_collaborators = pd.DataFrame(results["collaborators"])
    df_study_collaborators = pd.DataFrame(results["study_collaborators"])

    # conditionsModule
    df_conditions = pd.DataFrame(results["conditions"])
    df_study_conditions = pd.DataFrame(results["study_conditions"])
    df_keywords = pd.DataFrame(results["keywords"])
    df_study_keywords = pd.DataFrame(results["study_keywords"])

    # armsInterventionsModule
    df_arm_groups = pd.DataFrame(results["arm_groups"])
    df_arm_interventions = pd.DataFrame(results["arm_interventions"])
    df_interventions = pd.DataFrame(results["interventions"])
    df_study_interventions = pd.DataFrame(results["study_interventions"])
    df_other_intervention_names = pd.DataFrame(results["other_intervention_names"])
    df_study_intervention_aliases = pd.DataFrame(results["study_intervention_aliases"])

    # outcomesModule
    df_primary_outcomes = pd.DataFrame(results["primary_outcomes"])
    df_secondary_outcomes = pd.DataFrame(results["secondary_outcomes"])
    df_other_outcomes = pd.DataFrame(results["other_outcomes"])

    # contactsLocationsModule
    df_central_contacts = pd.DataFrame(results["central_contacts"])
    df_study_central_contacts = pd.DataFrame(results["study_central_contacts"])
    df_locations = pd.DataFrame(results["locations"])
    df_study_locations = pd.DataFrame(results["study_locations"])

    # referencesModule
    df_references = pd.DataFrame(results["references"])
    df_link_references = pd.DataFrame(results["link_references"])
    df_ipd_references = pd.DataFrame(results["ipd_references"])

    # outcomeMeasuresModule
    df_outcome_measures = pd.DataFrame(results["outcome_measures"])
    df_outcome_measure_groups = pd.DataFrame(results["outcome_measure_groups"])
    df_outcome_measure_denom_units = pd.DataFrame(
        results["outcome_measure_denom_units"]
    )
    df_outcome_measure_denom_counts = pd.DataFrame(
        results["outcome_measure_denom_counts"]
    )
    df_outcome_measure_groups_result = pd.DataFrame(
        results["outcome_measure_groups_result"]
    )
    df_outcome_measure_analyses = pd.DataFrame(results["outcome_measure_analyses"])
    df_outcome_measure_comparison_groups = pd.DataFrame(
        results["outcome_measure_comparison_groups"]
    )

    # participantFlowModule
    df_flow_groups = pd.DataFrame(results["flow_groups"])
    df_flow_periods = pd.DataFrame(results["flow_periods"])
    df_flow_period_milestones = pd.DataFrame(results["flow_period_milestones"])
    df_flow_period_milestone_achievements = pd.DataFrame(
        results["flow_period_milestone_achievements"]
    )
    df_flow_period_withdrawals = pd.DataFrame(results["df_flow_period_withdrawals"])
    df_flow_period_withdrawal_reasons = pd.DataFrame(
        results["flow_period_withdrawal_reasons"]
    )

    # adverseEventsModule
    df_adverse_events = pd.DataFrame(results["adverse_events"])
    df_event_groups = pd.DataFrame(results["event_groups"])
    df_serious_events = pd.DataFrame(results["serious_events"])
    df_serious_event_stats = pd.DataFrame(results["serious_event_stats"])
    df_other_events = pd.DataFrame(results["other_events"])
    df_other_event_stats = pd.DataFrame(results["other_event_stats"])

    # annotationModule
    df_violations = pd.DataFrame(results["violations"])

    # conditionBrowseModule
    df_conditions_mesh = pd.DataFrame(results["conditions_mesh"])
    df_study_conditions_mesh = pd.DataFrame(results["study_conditions_mesh"])
    df_conditions_mesh_ancestors = pd.DataFrame(results["conditions_mesh_ancestors"])
    df_study_conditions_mesh_ancestors = pd.DataFrame(
        results["study_conditions_mesh_ancestors"]
    )
    df_conditions_browse_leaves = pd.DataFrame(results["conditions_browse_leaves"])
    df_study_conditions_browse_leaves = pd.DataFrame(
        results["study_conditions_browse_leaves"]
    )
    df_conditions_browse_branches = pd.DataFrame(results["conditions_browse_branches"])
    df_study_conditions_browse_branches = pd.DataFrame(
        results["study_conditions_browse_branches"]
    )

    # interventionBrowseModule
    df_interventions_mesh = pd.DataFrame(results["interventions_mesh"])
    df_study_interventions_mesh = pd.DataFrame(results["study_interventions_mesh"])
    df_interventions_mesh_ancestors = pd.DataFrame(
        results["interventions_mesh_ancestors"]
    )
    df_study_interventions_mesh_ancestors = pd.DataFrame(
        results["study_interventions_mesh_ancestors"]
    )
    df_interventions_browse_leaves = pd.DataFrame(
        results["interventions_browse_leaves"]
    )
    df_study_interventions_browse_leaves = pd.DataFrame(
        results["study_interventions_browse_leaves"]
    )
    df_interventions_browse_branches = pd.DataFrame(
        results["interventions_browse_branches"]
    )
    df_study_interventions_browse_branches = pd.DataFrame(
        results["study_interventions_browse_branches"]
    )

    # dedupe

    return [
        df_studies,
        df_secondary_ids,
        df_nct_aliases,
        df_sponsors,
        df_study_sponsors,
        df_collaborators,
        df_study_collaborators,
        df_conditions,
        df_study_conditions,
        df_keywords,
        df_study_keywords,
        df_arm_groups,
        df_arm_interventions,
        df_interventions,
        df_study_interventions,
        df_other_intervention_names,
        df_study_intervention_aliases,
        df_primary_outcomes,
        df_secondary_outcomes,
        df_other_outcomes,
        df_central_contacts,
        df_study_central_contacts,
        df_locations,
        df_study_locations,
        df_references,
        df_link_references,
        df_ipd_references,
        df_outcome_measures,
        df_outcome_measure_groups,
        df_outcome_measure_denom_units,
        df_outcome_measure_denom_counts,
        df_outcome_measure_groups_result,
        df_outcome_measure_analyses,
        df_outcome_measure_comparison_groups,
        df_flow_groups,
        df_flow_periods,
        df_flow_period_milestones,
        df_flow_period_milestone_achievements,
        df_flow_period_withdrawals,
        df_flow_period_withdrawal_reasons,
        df_adverse_events,
        df_event_groups,
        df_serious_events,
        df_serious_event_stats,
        df_other_events,
        df_other_event_stats,
        df_violations,
        df_conditions_mesh,
        df_study_conditions_mesh,
        df_conditions_mesh_ancestors,
        df_study_conditions_mesh_ancestors,
        df_conditions_browse_leaves,
        df_study_conditions_browse_leaves,
        df_conditions_browse_branches,
        df_study_conditions_browse_branches,
        df_interventions_mesh,
        df_study_interventions_mesh,
        df_interventions_mesh_ancestors,
        df_study_interventions_mesh_ancestors,
        df_interventions_browse_leaves,
        df_study_interventions_browse_leaves,
        df_interventions_browse_branches,
        df_study_interventions_browse_branches,
    ]


In [44]:
def transform_studies_batch():
    try:
        batch_result = process_study_file("1.parquet")
        merged_batch_results = merge_batch_results(batch_result)
        dfs = post_process_tables(merged_batch_results)

        return dfs

    except Exception as e:
        raise


In [45]:
(
    df_studies,
    df_secondary_ids,
    df_nct_aliases,
    df_sponsors,
    df_study_sponsors,
    df_collaborators,
    df_study_collaborators,
    df_conditions,
    df_study_conditions,
    df_keywords,
    df_study_keywords,
    df_arm_groups,
    df_arm_interventions,
    df_interventions,
    df_study_interventions,
    df_other_intervention_names,
    df_study_intervention_aliases,
    df_primary_outcomes,
    df_secondary_outcomes,
    df_other_outcomes,
    df_central_contacts,
    df_study_central_contacts,
    df_locations,
    df_study_locations,
    df_references,
    df_link_references,
    df_ipd_references,
    df_outcome_measures,
    df_outcome_measure_groups,
    df_outcome_measure_denom_units,
    df_outcome_measure_denom_counts,
    df_outcome_measure_groups_result,
    df_outcome_measure_analyses,
    df_outcome_measure_comparison_groups,
    df_flow_groups,
    df_flow_periods,
    df_flow_period_milestones,
    df_flow_period_milestone_achievements,
    df_flow_period_withdrawals,
    df_flow_period_withdrawal_reasons,
    df_adverse_events,
    df_event_groups,
    df_serious_events,
    df_serious_event_stats,
    df_other_events,
    df_other_event_stats,
    df_violations,
    df_conditions_mesh,
    df_study_conditions_mesh,
    df_conditions_mesh_ancestors,
    df_study_conditions_mesh_ancestors,
    df_conditions_browse_leaves,
    df_study_conditions_browse_leaves,
    df_conditions_browse_branches,
    df_study_conditions_browse_branches,
    df_interventions_mesh,
    df_study_interventions_mesh,
    df_interventions_mesh_ancestors,
    df_study_interventions_mesh_ancestors,
    df_interventions_browse_leaves,
    df_study_interventions_browse_leaves,
    df_interventions_browse_branches,
    df_study_interventions_browse_branches,
    
) = transform_studies_batch()


<class 'list'>
1000


In [146]:
# print(f"SPONSORS {len(df_sponsors)}")
# df_sponsors = df_sponsors.drop_duplicates(subset=["sponsor_key"])
# print(f"DEDUPED SPONSORS {len(df_sponsors)}")

# print(f"STUDY SPONSORS {len(df_study_sponsors)}")
# df_study_sponsors = df_study_sponsors.drop_duplicates(subset=["sponsor_key", "study_key"])
# print(f"DEDUPED STUDY SPONSORS {len(df_study_sponsors)}")
# print("----------------")

# print(f"CONDITIONS {len(df_conditions)}")
# df_conditions = df_conditions.drop_duplicates(subset=["condition_key"])
# print(f"DEDUPED CONDITIONS {len(df_conditions)}")

# print(f"STUDY CONDITIONS {len(df_study_conditions)}")
# df_study_conditions = df_study_conditions.drop_duplicates(subset=["condition_key", "study_key"])
# print(f"DEDUPED STUDY CONDITIONS {len(df_study_conditions)}")
# print("----------------")

# print(f"KEYWORDS {len(df_keywords)}")
# df_keywords = df_keywords.drop_duplicates(subset=["keyword_key"])
# print(f"DEDUPED KEYWORDS {len(df_keywords)}")

# print(f"STUDY KEYWORDS {len(df_study_keywords)}")
# df_study_keywords = df_study_keywords.drop_duplicates(subset=["keyword_key", "study_key"])
# print(f"DEDUPED STUDY KEYWORDS {len(df_study_keywords)}")
# print("----------------")


# print(f"INTERVENTIONS {len(df_interventions)}")
# df_interventions = df_interventions.drop_duplicates(subset=["intervention_key"])
# print(f"DEDUPED INTERVENTIONS {len(df_interventions)}")
# print("----------------")

# print(f"INTERVENTIONS OTHER {len(df_other_intervention_names)}")
# df_other_intervention_names = df_other_intervention_names.drop_duplicates(subset=["intervention_key"])
# print(f"DEDUPED OTHER INTERVENTIONS {len(df_other_intervention_names)}")
# print("----------------")


# print(f"STUDY INTERVENTIONS {len(df_study_interventions)}")
# df_study_interventions = df_study_interventions.drop_duplicates(subset=["intervention_key", "study_key"])
# print(f"DEDUPED STUDY INTERVENTIONS {len(df_study_interventions)}")
# print("----------------")


# print(f"STUDY INTERVENTION ALIASES {len(df_study_intervention_aliases)}")

# df_study_intervention_aliases = df_study_intervention_aliases.drop_duplicates(subset=["intervention_key", "study_key"])
# print(f"DEDUPED STUDY INTERVENTION ALIASES {len(df_study_intervention_aliases)}")
# print("----------------")


# print(f"Arm groups {len(df_arm_groups)}")
# df_arm_groups = df_arm_groups.drop_duplicates(subset=["study_key", "arm_group_key"])
# print(f"DEDUPED Arm groups {len(df_arm_groups)}")


# print(f"ARM GROUPS INTERVENTION {len(df_arm_interventions)}")
# df_arm_interventions = df_arm_interventions.drop_duplicates(subset=["arm_intervention_key", "study_key", "arm_group_key"])
# print(f"DEDUPED ARM GROUPS INTERVENTION {len(df_arm_interventions)}")
# print("----------------")


# print(f"primary outcomes {len(df_primary_outcomes)}")
# df_primary_outcomes = df_primary_outcomes.drop_duplicates(subset=["outcome_key"])
# print(f"DEDUPED primary outcomes {len(df_primary_outcomes)}")

# print(f"secondary outcomes {len(df_secondary_outcomes)}")
# df_secondary_outcomes = df_secondary_outcomes.drop_duplicates(subset=["outcome_key"])
# print(f"DEDUPED secondary outcome {len(df_secondary_outcomes)}")

# print(f"other outcomes {len(df_other_outcomes)}")
# df_other_outcomes = df_other_outcomes.drop_duplicates(subset=["outcome_key"])
# print(f"DEDUPED other outcomes {len(df_other_outcomes)}")


# print(f"CONTACTS {len(df_central_contacts)}")
# df_central_contacts = df_central_contacts.drop_duplicates(subset=["contact_key"])
# print(f" DEDUPED CONTACTS {len(df_central_contacts)}")

# print(f"STUDY CONTACTS {len(df_study_central_contacts)}")
# df_study_central_contacts = df_study_central_contacts.drop_duplicates(subset=["contact_key", "study_key"])
# print(f"DEDUPED STUDY CONTACTS {len(df_study_central_contacts)}")


# print(f"Locations {len(df_locations)}")
# df_locations = df_locations.drop_duplicates(subset=["location_key"])
# print(f" DEDUPED Locations {len(df_locations)}")


# print("----------------")

# print(f"REFERENCES {len(df_references)}")
# df_references = df_references.drop_duplicates(subset=["ref_key"])
# print(f"DEDUPED REFERENCES {len(df_references)}")


# print(f"LINKS {len(df_link_references)}")
# df_link_references = df_link_references.drop_duplicates(subset=["link_key"])
# print(f"LINKS {len(df_link_references)}")

# print("----------------")
# print(f"IPDS {len(df_ipd_references)}")
# df_ipd_references = df_ipd_references.drop_duplicates(subset=["ipd_key"])
# print(f"DEDUPED IPDS {len(df_ipd_references)}")







# print("----------------")
# print(f"condition mesh {len(df_conditions_mesh)}")
# df_conditions_mesh = df_conditions_mesh.drop_duplicates(subset=["mesh_key"])
# print(f"DEDUPED condition mesh {len(df_conditions_mesh)}")

# print("----------------")
# print(f"study condition mesh {len(df_study_conditions_mesh)}")
# df_study_conditions_mesh = df_study_conditions_mesh.drop_duplicates(subset=["mesh_key", "study_key"])
# print(f"DEDUPED study condition mesh {len(df_study_conditions_mesh)}")


# print("----------------")
# print(f"condition mesh ancestors {len(df_conditions_mesh_ancestors)}")
# df_conditions_mesh_ancestors = df_conditions_mesh_ancestors.drop_duplicates(subset=["ancestor_key"])
# print(f"DEDUPED condition mesh ancestors {len(df_conditions_mesh_ancestors)}")

# print("----------------")
# print(f"study condition mesh ancestors {len(df_study_conditions_mesh_ancestors)}")
# df_study_conditions_mesh_ancestors = df_study_conditions_mesh_ancestors.drop_duplicates(subset=["ancestor_key", "study_key"])
# print(f"DEDUPED study condition mesh ancestors {len(df_study_conditions_mesh_ancestors)}")


# duplicates = df_study_conditions_mesh[
#     df_study_conditions_mesh.duplicated(subset=["mesh_key", "study_key"], keep=False)
# ].sort_values(["mesh_key", "study_key"])

# print(duplicates.head(10))
# print(df_study_interventions.groupby(['study_key', 'intervention_key']).size().sort_values(ascending=False).head(20))



In [27]:
# print(df_studies.head())

In [151]:
df_studies.to_csv("data/study_data.csv", index=False)

In [152]:
df_secondary_ids.to_csv("data/secondary_ids.csv", index=False)
df_nct_aliases.to_csv("data/nct_aliases.csv", index=False)

In [153]:
df_sponsors.to_csv("data/sponsors.csv", index=False)
df_study_sponsors.to_csv("data/bridge_study_sponsors.csv", index=False)
df_collaborators.to_csv("data/collab.csv", index=False)
df_study_collaborators.to_csv("data/bridge_study_collab.csv", index=False)

In [23]:
df_conditions.to_csv("data/conditions.csv", index=False)
df_study_conditions.to_csv("data/study_conditions.csv", index=False)
df_keywords.to_csv("data/keywords.csv", index=False)
df_study_keywords.to_csv("data/study_keywords.csv", index=False)

In [90]:
df_arm_groups.to_csv("data/arms.csv", index=False)
df_arm_interventions.to_csv("data/arm_interventions.csv", index=False)
df_interventions.to_csv("data/interventions.csv", index=False)
df_study_interventions.to_csv("data/study_interventions.csv", index=False)
df_other_intervention_names.to_csv("data/other_intervention_names.csv", index=False)
df_study_intervention_aliases.to_csv("data/study_intervention_aliases.csv", index=False)

In [117]:
df_primary_outcomes.to_csv("data/df_primary_outcomes.csv", index=False)
df_secondary_outcomes.to_csv("data/df_secondary_outcomes.csv", index=False)
df_other_outcomes.to_csv("data/df_other_outcomes.csv", index=False)

In [119]:
df_central_contacts.to_csv("data/contacts.csv", index=False)
df_study_central_contacts.to_csv("data/study_contacts.csv", index=False)
df_study_locations.to_csv("data/study_locations.csv", index=False)
df_locations.to_csv("data/locations.csv", index=False)

In [104]:
df_references.to_csv("data/refs.csv", index=False)
df_link_references.to_csv("data/links.csv", index=False)
df_ipd_references.to_csv("data/ipds.csv", index=False)

In [46]:
df_outcome_measures.to_csv("data/df_outcome_measures.csv", index=False)
df_outcome_measure_groups.to_csv("data/df_outcome_measure_groups.csv", index=False)
df_outcome_measure_denom_units.to_csv("data/df_outcome_measure_denom_units.csv", index=False)
df_outcome_measure_denom_counts.to_csv("data/df_outcome_measure_denom_counts.csv", index=False)
df_outcome_measure_groups_result.to_csv("data/df_outcome_measure_groups_result.csv", index=False)
df_outcome_measure_analyses.to_csv("data/df_outcome_measure_analyses.csv", index=False)
df_outcome_measure_comparison_groups.to_csv("data/df_outcome_measure_comparison_groups.csv", index=False)

In [47]:
df_flow_groups.to_csv("data/df_flow_groups.csv", index=False) 
df_flow_periods.to_csv("data/df_flow_periods.csv", index=False) 
df_flow_period_milestones.to_csv("data/df_flow_period_milestones.csv", index=False) 
df_flow_period_milestone_achievements.to_csv("data/df_flow_period_milestone_achievements.csv", index=False) 
df_flow_period_withdrawals.to_csv("data/df_flow_period_withdrawals.csv", index=False) 
df_flow_period_withdrawals.to_csv("data/df_flow_period_withdrawals.csv", index=False) 
df_flow_period_withdrawal_reasons.to_csv("data/df_flow_period_withdrawal_reasons.csv", index=False) 


In [148]:
df_adverse_events.to_csv("data/df_adverse_events.csv", index=False)
df_event_groups.to_csv("data/df_event_groups.csv", index=False) 
df_serious_events.to_csv("data/df_serious_events.csv", index=False) 
df_serious_event_stats.to_csv("data/df_serious_event_stats.csv", index=False) 
df_other_events.to_csv("data/df_other_events.csv", index=False) 
df_other_event_stats.to_csv("data/df_other_event_stats.csv", index=False) 

In [147]:
df_violations.to_csv("data/df_violations.csv", index=False) 

In [176]:
df_conditions_mesh.to_csv("data/df_conditions_mesh.csv", index=False) 
df_study_conditions_mesh.to_csv("data/df_study_conditions_mesh.csv", index=False) 
df_conditions_mesh_ancestors.to_csv("data/df_conditions_mesh_ancestors.csv", index=False) 
df_study_conditions_mesh_ancestors.to_csv("data/df_study_conditions_mesh_ancestors.csv", index=False) 
df_conditions_browse_leaves.to_csv("data/df_conditions_browse_leaves.csv", index=False) 
df_study_conditions_browse_leaves.to_csv("data/df_study_conditions_browse_leaves.csv", index=False) 
df_conditions_browse_branches.to_csv("data/df_conditions_browse_branches.csv", index=False) 
df_study_conditions_browse_branches.to_csv("data/df_study_conditions_browse_branches.csv", index=False) 

In [177]:
df_interventions_mesh.to_csv("data/df_interventions_mesh.csv", index=False) 
df_study_interventions_mesh.to_csv("data/df_study_interventions_mesh.csv", index=False) 
df_interventions_mesh_ancestors.to_csv("data/df_interventions_mesh_ancestors.csv", index=False) 
df_study_conditions_mesh_ancestors.to_csv("data/df_study_conditions_mesh_ancestors.csv", index=False) 
df_interventions_browse_leaves.to_csv("data/df_interventions_browse_leaves.csv", index=False) 
df_study_interventions_browse_leaves.to_csv("data/df_study_interventions_browse_leaves.csv", index=False) 
df_interventions_browse_branches.to_csv("data/df_interventions_browse_branches.csv", index=False) 
df_study_interventions_browse_branches.to_csv("data/df_study_interventions_browse_branches.csv", index=False) 

In [21]:
# duplicates.to_csv("data/dups.csv", index=False)

In [100]:
df_studies[df_studies['study_key'] == '49387ab78594fffd']

,study_key,nct_id,brief_title,official_title,acronym,org_study_id,brief_summary,detailed_desc,responsible_party,study_type,...,flow_type_units_analysed,certain_agreement_pi_sponsor_employee,certain_agreement_restrictive,certain_agreement_other_details,certain_agreement_restriction_type,sub_tracking_estimated_results_date,version_holder,has_results,last_updated,limitations_desc
688,49387ab78594fffd,NCT00138086,Yttrium-90 Ibritumomab Tiuxetan (Zevalin) With...,Targeted Intensification by a New Preparative ...,None,Z-BEAM,The objective of this study is to evaluate the...,The indolent course of the low-grade B-cell ly...,NaN,INTERVENTIONAL,...,NaN,NaN,NaN,NaN,NaN,NaN,2026-01-08,False,2006-09-08,NaN
